In [42]:
import numpy
import cv2
import cv2.aruco as aruco
# Create ChArUco board
gridboard = aruco.CharucoBoard_create(
        squaresX=6, 
        squaresY=8, 
        squareLength=0.04, 
        markerLength=0.02, 
        dictionary=aruco.Dictionary_get(aruco.DICT_5X5_50))

img = gridboard.draw(outSize=(988, 1400))
cv2.imwrite("test_charuco.jpg", img)
cv2.imshow('Gridboard', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import numpy as np
import cv2
import cv2.aruco as aruco
# Create ArUco marker
aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_50)
img = aruco.drawMarker(aruco_dict, 4, 200)
cv2.imwrite("test_marker4.jpg", img)
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import numpy
import cv2
from cv2 import aruco
import argparse
import pickle
import glob
import os

ARUCO_DICT=aruco.Dictionary_get(aruco.DICT_5X5_50)
CHARUCO_BOARD = aruco.CharucoBoard_create(squaresX=6,squaresY=8,squareLength=0.04,markerLength=0.02,dictionary=ARUCO_DICT)

captures=50
#num of squares for a valid calibration image
validresponse=20
corners_all = []
ids_all = [] 
image_size = None 

cap = cv2.VideoCapture('123.mp4')
validCaptures = 0

while cap.isOpened():
    ret, img = cap.read()
    if ret is False:
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(
        image=gray,
        dictionary=ARUCO_DICT)
    if ids is None:
        continue
    img = aruco.drawDetectedMarkers(image=img,corners=corners)
    response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
        markerCorners=corners,
        markerIds=ids,
        image=gray,
        board=CHARUCO_BOARD)
    if response > validresponse:
        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)
        # If our image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]
        # Reproportion the image, maxing width or height at 1000
        proportion = max(img.shape) / 1000.0
        img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
        validCaptures += 1
        if validCaptures == captures:
            break
cv2.destroyAllWindows()
print("{} valid captures".format(validCaptures))

# Now that we've seen all of our images, perform the camera calibration
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
    charucoCorners=corners_all,
    charucoIds=ids_all,
    board=CHARUCO_BOARD,
    imageSize=image_size,
    cameraMatrix=None,
    distCoeffs=None)

print("Camera intrinsic parameters matrix:\n{}".format(cameraMatrix))
print("\nCamera distortion coefficients:\n{}".format(distCoeffs))

f = open('./CameraCalibration.pckl', 'wb')
pickle.dump((cameraMatrix, distCoeffs, rvecs, tvecs), f)
f.close()

print('Calibration successful. Calibration file created: {}'.format('CameraCalibration.pckl'))

50 valid captures
Camera intrinsic parameters matrix:
[[8.29978089e+03 0.00000000e+00 8.43156528e+02]
 [0.00000000e+00 1.20816386e+04 3.49463376e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Camera distortion coefficients:
[[-2.24940883e+03 -2.13844881e+00  3.11717067e+01  8.91861279e-01
   3.18580580e-04]]
Calibration successful. Calibration file created: CameraCalibration.pckl


In [13]:
import numpy as np
import cv2
import cv2.aruco as aruco
import os
import pickle

f = open('./CameraCalibration.pckl', 'rb')
(cameraMatrix, distCoeffs, rvecs, tvecs) = pickle.load(f)
f.close()

ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_50)

cam = cv2.VideoCapture(0)
cam.set(3, 1280)
cam.set(4, 720)

while(cam.isOpened()):
    ret,img = cam.read()
    if ret == True:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
        img = aruco.drawDetectedMarkers(img, corners, borderColor=(0, 0, 255))
        if ids is not None and len(ids) > 0:
            rotation_vectors, translation_vectors, _objPoints = aruco.estimatePoseSingleMarkers(corners, 1, cameraMatrix, distCoeffs)
            for rvec, tvec in zip(rotation_vectors, translation_vectors):
                img = cv2.drawFrameAxes(img, cameraMatrix, distCoeffs, rvec, tvec, 0.5)
    cv2.imshow('img',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam.release()
        break
    
cv2.destroyAllWindows()

# Custom marker

In [22]:
import numpy as np
import cv2
cv2.imwrite("marker.png", cv2.resize(np.array([
    [0, 0,   0,   0,   0,   0,   0,   0],
    [0, 0,   255, 0,   0,   255, 0,   0],
    [0, 0,   255, 255, 255, 255, 255, 0],
    [0, 255, 255, 255, 255, 255, 0,   0],
    [0, 255, 255, 255, 255, 255, 255, 0],
    [0, 255, 255, 255, 255, 255, 255, 0],
    [0, 0,   0,   0,   255, 0,   255, 0],
    [0, 0,   0,   0,   0,   0,   0,   0],
]), (512, 512), interpolation=cv2.INTER_NEAREST))

True

In [1]:
import numpy as np
import math
import cv2
import cv2.aruco as aruco
import copy
import imutils
import pickle
#размер маркера
mrk_size = 200
minArea=20
pi=3.14159
len_marker=125
#углы маркера
edgM = np.array([
    [0, 0],
    [mrk_size-1, 0],
    [mrk_size-1, mrk_size-1],
    [0, mrk_size-1]], dtype="float32")
#пока библиотека состоит из 1 маркера                 
dict_cust_marker=[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
#определение подходящих контуров  
def get_contour(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray, (13, 13), 0) 
    ret, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY) 
    contour_list = []
    contours,h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    index=[]
    polygons=[]
    quads=[]
    if len(contours)>0:
        for i in h[0]:
            if i[3]!=-1:
                index.append(i[3])
        for i in index:
            polygons.append(cv2.approxPolyDP(contours[i], 3, True))
        for i in polygons:
            if len(i)==4 and cv2.contourArea(i)>=minArea:
                quads.append(i) 
    return quads



#расшифровка маркера
def ident(tag):
    num_rot=0
    for i in range(2):
        for j in range(2):
            if (tag[37+i*125,37+j*125]==255):
                n=i*2+j
                if (n==0):
                    num_rot=0
                elif (n==1):
                    num_rot=1
                elif (n==2):
                    num_rot=3
                elif (n==3):
                    num_rot=2
    (h, w) = tag.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    M =cv2.getRotationMatrix2D((cX, cY), num_rot*90, 1.0)
    rotated = cv2.warpAffine(tag, M, (w, h))
    id_code=[]
    for i in range(6):
        for j in range(6):
            if (((i==0 or i==5) and (j==0 or j==5)) or ((i!=0 and i!=5) and (j!=0 and j!=5))):
                continue
            id_code.append(int(rotated[37+i*25,37+j*25]<225))
    return id_code,num_rot,rotated   

    
def draw_axis(img,points,mtx,dist,num_rot):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgp = np.array(points[:,0], dtype="float32")
    objp = np.array([[0.,0.,0.],
                     [1.,0.,0.],
                     [1.,1.,0.],
                     [0.,1.,0.]], dtype="float32")  
    axis = np.float32([[0,0,0], [1,0,0], [0,1,0], [0,0,-1]])
    objp = np.roll(objp, num_rot, axis=0)
    cv2.cornerSubPix(gray,imgp,(11,11),(-1,-1),criteria)
    _, rvecs, tvecs, inliers  = cv2.solvePnPRansac(objp, imgp, mtx, dist)
    imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    imgpts = np.int32(imgpts).reshape(-1,2)
    for i in range(3):
            cv2.line(img,tuple(imgpts[0]),tuple(imgpts[i+1]),(255,0,0),4)
    return img, rvecs     
#определение угла по стержню
def high_accuracy(tag):
    tag1=tag[50:150,50:150]
    params = cv2.SimpleBlobDetector_Params()
    params.filterByCircularity = True;
    params.minCircularity = 0.85;
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(tag1)
    pts = cv2.KeyPoint_convert(keypoints)
    angle=[]
    for p in pts:
        angle.append(np.arcsin((50-p)/len_marker)*180/pi)
    return(angle)

# Использование камеры 

In [57]:
cam = cv2.VideoCapture(0)
cam.set(3, 1280)
cam.set(4, 720)
f = open('./CameraCalibration.pckl', 'rb')
(cameraMatrix, distCoeffs, rvecs, tvecs) = pickle.load(f)
f.close()
while(cam.isOpened()):
    ret,img = cam.read()
    if ret == False:
        cam.release()
        cv2.destroyAllWindows()
        break
    contour_list=get_contour(img)
    if contour_list==None or contour_list==0:
        continue
    else:
        decoded=[]
        num_rot=[]
        for i in range(len(contour_list)):
            c_rez = contour_list[i][:, 0]
            H_matrix, status = cv2.findHomography(c_rez,edgM)
            tag = cv2.warpPerspective(img, H_matrix, (mrk_size, mrk_size))
            tag = cv2.cvtColor(tag, cv2.COLOR_BGR2GRAY)
            tag1, thresh = cv2.threshold(tag, 100, 255, cv2.THRESH_BINARY)
            decoded,num_rot=ident(thresh)
            

            M =cv2.getRotationMatrix2D((cX, cY), -num_rot*90, 1.0)
            rotated = cv2.warpAffine(tag, M, (w, h))
            
            a=high_accuracy(rotated)
               
            ##
            if a != []:
                text= "x = %d[deg],y = %d[deg]"%(a[0][0],a[0][1])
                fontFace = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                thickness = 1
                size, baseLine = cv2.getTextSize(text, fontFace, fontScale, thickness)
                cv2.putText(img, text, (0, size[1]), fontFace, fontScale,(0, 0, 255), thickness)
            ##
        if len(contour_list)>0:
            for i in contour_list:
                img=draw_axis(img,i,cameraMatrix,distCoeffs)
        
    
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam.release()
        cv2.destroyAllWindows()
        break    

# Использование экрана ПК


In [4]:
import numpy as np
import cv2
from mss import mss
from PIL import Image
import os
import pickle

f = open('CameraCalibration.pckl', 'rb')
(cameraMatrix, distCoeffs, rvecs, tvecs) = pickle.load(f)
f.close()

mon = {'top': 400, 'left': 400, 'width': 500, 'height': 500}

sct = mss()

while 1:
    sct.get_pixels(mon)
    img = np.array(Image.frombytes('RGB', (sct.width, sct.height), sct.image))
    
    contour_list=get_contour(img)
    correct_contour=[]
    if contour_list==None or contour_list==0:
        continue
    else:
        decode=[]
        num_rot=[]
        for i in range(len(contour_list)):
            c_rez = contour_list[i][:, 0]
            H_matrix, status = cv2.findHomography(c_rez,edgM)
            tag = cv2.warpPerspective(img, H_matrix, (mrk_size, mrk_size))
            tag = cv2.cvtColor(tag, cv2.COLOR_BGR2GRAY)
            tag1, thresh = cv2.threshold(tag, 100, 255, cv2.THRESH_BINARY)
            code,num,rotated=ident(thresh)
            decode.append(code)
            num_rot.append(num)
            if code==dict_cust_marker:
                correct_contour.append(contour_list[i])
                a=high_accuracy(rotated)
                if a != []:
                    text= "x = %d[deg],y = %d[deg]"%(a[0][0],a[0][1])
                    #text = "Rotation: %d[deg], Code:" % (90 * rot90)  + ''.join(str(e) for e in decoded)
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 0.5
                    thickness = 1
                    
                    
                    
                    size1, baseLine1 = cv2.getTextSize(text, fontFace, fontScale, thickness)
                    text_coord = (5,baseLine1+5)
                    cv2.rectangle(img,(text_coord[0]-5, text_coord[1]+baseLine1),(size1[0]+100, 0),(255, 255, 255),-1)
                    cv2.putText(img, text, (0, size1[1]), fontFace, fontScale,(0, 0, 255), thickness)
             
            ##

            ##
        if len(correct_contour)>0:
            indx=0
            for i in correct_contour:
                img,rvecs=draw_axis(img,i,cameraMatrix,distCoeffs,num_rot[indx])
                indx=indx+1
                if rvecs.size > 0:
                    a=[]
                    for i in range(len(rvecs)):
                        a.append(int(rvecs[i][0]*180/pi))
                    #text= "x = %d[deg],y = %d[deg],z = %d[deg]"%(an[0],an[1],a[2])
                    text= "z = %d[deg]"%(a[2])
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 0.5
                    thickness = 1
                    
                    size, baseLine = cv2.getTextSize(text, fontFace, fontScale, thickness)
                    text_coord = (0,baseLine*3+5)
                    #cv2.rectangle(img,(baseLine1, size[1]*2-5),(size[0]+5, size[1]*3+5),(255, 0, 255),-1)
                    
                    cv2.putText(img, text, (baseLine1+220, size1[1]), fontFace, fontScale,(0, 0, 255), thickness)
                    
                    #                    size, baseLine = cv2.getTextSize(text, fontFace, fontScale, thickness)
                    #text_coord = (0,baseLine*3+5)
                    #cv2.rectangle(img,(0, size[1]*2-5),(size[0]+5, size[1]*3+5),(255, 255, 255),-1)
                    
                    #cv2.putText(img, text, (0, size[1]*3), fontFace, fontScale,(0, 0, 255), thickness)
        #img = cv2.drawContours(img, contour_list,  -1, (0, 255, 0), 2)

    
    
    
    cv2.imshow('test', img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break